Assignment: SQL Notebook for Peer Assignment
Estimated time needed: 60 minutes.

Introduction
Using this Python notebook you will:

Understand the Spacex DataSet
Load the dataset into the corresponding table in a Db2 database
Execute SQL queries to answer assignment questions
Overview of the DataSet
SpaceX has gained worldwide attention for a series of historic milestones.

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010. SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage.

Therefore if we can determine if the first stage will land, we can determine the cost of a launch.

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.







Download the datasets

In [1]:
!pip install sqlalchemy==1.3.9

In [2]:
%load_ext sql

Let us first load the SQL extension and establish a connection with the database

In [3]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [4]:
!pip install -q pandas==1.1.5

In [5]:
%sql sqlite:///my_data1.db

'Connected: @my_data1.db'

In [6]:
import pandas as pd 

In [7]:
df = pd.read_csv('Spacex.csv')

In [8]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
(sqlite3.OperationalError) table SPACEXTABLE already exists
[SQL: create table SPACEXTABLE as select * from SPACEXTBL where Date is not null]
(Background on this error at: http://sqlalche.me/e/e3q8)


Task 1. Display the names of the unique launch sites in the space mission

In [9]:
%sql SELECT COUNT(DISTINCT LAUNCH_SITE) FROM SPACEXTBL;

 * sqlite:///my_data1.db
Done.


COUNT(DISTINCT LAUNCH_SITE)
4


In [10]:
%sql SELECT DISTINCT LAUNCH_SITE FROM SPACEXTBL;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


Task 2. Display 5 records where launch sites begin with the string 'CCA'

In [11]:
%sql SELECT * FROM SPACEXTBL WHERE (LAUNCH_SITE) LIKE 'CCA%' LIMIT 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
6/4/2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
12/8/2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
22/05/2012,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
10/8/2012,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
3/1/2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


Task 3. Display the total payload mass carried by boosters launched by NASA (CRS)

In [12]:
%sql select sum(PAYLOAD_MASS__KG_) as total_payload_mass from SPACEXTBL where customer = 'NASA (CRS)';

 * sqlite:///my_data1.db
Done.


total_payload_mass
45596


Task 4. Display average payload mass carried by booster version F9 v1.1

In [13]:
%sql select avg(PAYLOAD_MASS__KG_) as Average_payload_mass from SPACEXTBL where Booster_Version = 'F9 v1.1';

 * sqlite:///my_data1.db
Done.


Average_payload_mass
2928.4


Task 5. List the date when the first succesful landing outcome in ground pad was acheived.
Hint:Use min function

In [14]:
%sql select min(Date) as 'first succesful landing outcome'from SPACEXTBL where Landing_Outcome LIKE 'Sucess (ground pad)';

 * sqlite:///my_data1.db
Done.


first succesful landing outcome
None


In [15]:
%sql select min(DATE) from SPACEXTBL;

 * sqlite:///my_data1.db
Done.


min(DATE)
1/10/2015


In [16]:
# This is the full record of the first successful landing outcome in ground pad
%sql SELECT * FROM SPACEXTBL WHERE DATE = '1/10/2015';

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
1/10/2015,9:47:00,F9 v1.1 B1012,CCAFS LC-40,SpaceX CRS-5,2395,LEO (ISS),NASA (CRS),Success,Failure (drone ship)


Task 6. List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [17]:
%sql select Booster_Version from SPACEXTBL where Landing_Outcome='Success (drone ship)' and PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000;

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


Task 7. List the total number of successful and failure mission outcomes

In [21]:
%sql select count(*) as Mission_Outcome from SPACEXTBL where Landing_Outcome NOT LIKE '%success%' OR Landing_Outcome LIKE '%success%';

 * sqlite:///my_data1.db
Done.


Mission_Outcome
101


In [22]:
%sql select count(*) as Mission_Outcome from SPACEXTBL where Landing_Outcome NOT LIKE '%success%'

 * sqlite:///my_data1.db
Done.


Mission_Outcome
40


In [23]:
%sql select count(*) as Mission_Outcome from SPACEXTBL where Landing_Outcome LIKE '%success%'

 * sqlite:///my_data1.db
Done.


Mission_Outcome
61


In [24]:
%sql select count(MISSION_OUTCOME) as missionoutcomes from SPACEXTBL GROUP BY MISSION_OUTCOME;

 * sqlite:///my_data1.db
Done.


missionoutcomes
1
98
1
1


List the names of the booster_versions which have carried the maximum payload mass. Use a subquery

In [25]:
%sql select Booster_Version FROM SPACEXTBL WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTBL);

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


In [26]:
# Maximum Payload Mass (kg)
%sql SELECT max(PAYLOAD_MASS__KG_) AS "Max Payload Mass (kg)" FROM SPACEXTBL;

 * sqlite:///my_data1.db
Done.


Max Payload Mass (kg)
15600


In [27]:
%sql SELECT Booster_Version, PAYLOAD_MASS__KG_ FROM SPACEXTBL WHERE PAYLOAD_MASS__KG_ = (SELECT max(PAYLOAD_MASS__KG_) FROM SPACEXTBL);


 * sqlite:///my_data1.db
Done.


Booster_Version,PAYLOAD_MASS__KG_
F9 B5 B1048.4,15600
F9 B5 B1049.4,15600
F9 B5 B1051.3,15600
F9 B5 B1056.4,15600
F9 B5 B1048.5,15600
F9 B5 B1051.4,15600
F9 B5 B1049.5,15600
F9 B5 B1060.2,15600
F9 B5 B1058.3,15600
F9 B5 B1051.6,15600


Task 9
List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.
Note: SQLLite does not support monthnames. So you need to use substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.

In [28]:
%sql SELECT count(DISTINCT Date) AS "Number of Unique Launch Dates" FROM SPACEXTBL;

 * sqlite:///my_data1.db
Done.


Number of Unique Launch Dates
101


In [30]:
%sql SELECT DATE, Year(DATE) AS "Year", MONTH(DATE) AS "Month", DAY(DATE) AS "Day", DAYOFWEEK(DATE) AS "day_of_week" FROM SPACEXTBL LIMIT 5;

 * sqlite:///my_data1.db
(sqlite3.OperationalError) no such function: Year
[SQL: SELECT DATE, Year(DATE) AS "Year", MONTH(DATE) AS "Month", DAY(DATE) AS "Day", DAYOFWEEK(DATE) AS "day_of_week" FROM SPACEXTBL LIMIT 5;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [34]:
%sql SELECT MONTHNAME(DATE) AS "Month", Landing__Outcome, Booster_Version, Launch_Site  FROM SPACEXTBL WHERE Landing__Outcome = 'Failure (drone ship)' AND YEAR(DATE) = 2015;


 * sqlite:///my_data1.db
(sqlite3.OperationalError) no such function: MONTHNAME
[SQL: SELECT MONTHNAME(DATE) AS "Month", Landing__Outcome, Booster_Version, Launch_Site  FROM SPACEXTBL WHERE Landing__Outcome = 'Failure (drone ship)' AND YEAR(DATE) = 2015;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [ ]:
%sql SELECT CASE  WHEN substr(Date, 6, 2) = 'months'

Task 10
Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.

In [38]:
%sql SELECT Landing_Outcome, count(Landing_Outcome) AS "Count" FROM SPACEXTBL WHERE (Landing_Outcome) LIKE '%success%' and DATE BETWEEN '2010-06-04' AND '2017-03-20' GROUP BY Landing_Outcome ORDER BY count(Landing_Outcome) DESC;


 * sqlite:///my_data1.db
Done.


Landing_Outcome,Count


In [39]:
%sql SELECT Landing_Outcome, count(Landing_Outcome) AS "Count" FROM SPACEXTBL WHERE DATE BETWEEN '2010-06-04' AND '2017-03-20' GROUP BY Landing_Outcome ORDER BY count(Landing_Outcome) DESC;

 * sqlite:///my_data1.db
Done.


Landing_Outcome,Count
